<center><h1>Quantum Harmonic Oscillator</h1></center>

### Importing the necessary modules and specifying the boundary conditions

In [ ]:
import sys
import vpython as vp
from math import *
sys.path.append('/home/Tesla/Programming/PrOjEcT/Modules')
from RK4CD_module import * 

# Initial conditions to solve the ODE
psi0 = 0.0
phi0 = 1.0
scene = vp.canvas()

### List of functions for the Project

In [ ]:
def V(x):
    '''
    Returns the Potential 
    '''
    return x**2
    
def f2(x,phi,psi):
    return phi

def f1(x,phi,psi):
    global E
    return (V(x)-E)*psi

def solver(e):
    '''
    Returns the right boundary value of the wavefunction for a given arbitrary energy
    '''
    global E
    E = e
    A = 0.0
    psi,phi,x = ODE_runge_4th_CD(f1,f2,phi0,psi0,-5,5)
    for i in range(len(psi)):
        A += psi[i]**2
    for i in range(len(psi)):
        psi[i] = psi[i]/sqrt(1.0*A)
    return psi[-1]

def solver_fullfunc(e):
    '''
    Returns the Wavefunction for a given energy
    '''
    global E
    E = e
    A = 0.0
    psi,phi,x = ODE_runge_4th_CD(f1,f2,phi0,psi0,-5,5)
    for i in range(len(psi)):
        A += psi[i]**2
    for i in range(len(psi)):
        psi[i] = psi[i]/sqrt(1.0*A)
    return psi,x

### Initializing variables and defining graph objects in VPython

In [ ]:
E_min = 0.0
E_max = 15.0
dE = 1.0
Edash = linspace(E_min,E_max,(E_max-E_min)/dE+1)
boundary = []

for e in Edash:
    boundary.append(solver(e))
e1,e2,wf1,wf2,PSIS,EIGEN,PSI_Im,PSI_Re = ([] for _ in range(8))

gr = vp.graph(xmin = -6.0, xmax = 6.0, ymin = -0.085, ymax = 0.085, title = 'Time Evolution', xtitle = 'x')
psireal = vp.gcurve(color = vp.color.red, label = 'Re(psi)')
psiimag = vp.gcurve(color = vp.color.green, label = 'Im(psi)')
psisquare = vp.gcurve(color = vp.color.blue, label = 'psi square')

### Extracting the eigenvalues of the system by Bisection method

In [ ]:
for i in range(len(Edash)-1):
    if boundary[i]*boundary[i+1]<=0:
        e1.append(Edash[i]);e2.append(Edash[i+1])
        wf1.append(boundary[i]);wf2.append(boundary[i+1])
        
for i in range(len(e1)):
    x0 = e1[i]
    x1 = e2[i]
    dx = x1 - x0
    c = 0
    while(abs(dx)>0.00000001):
        c = (x0+x1)/2
        if(solver(c)==0):
            break
        if(solver(x0)*solver(c)<0):
            x1 = c
        else:
            x0 = c
        dx = x1 - x0
        psi,x = solver_fullfunc(c)
    PSIS.append(psi)
    plot(x,psi)
    EIGEN.append(c)
    print("Eigenvalue no %lf:%lf"%(i+1,c))
show()

### Picking up arbitrary c's and finding the time dependent state

In [ ]:
c1 = 0.5
c2 = 0.5
c3 = 0.5
c4 = 0.5
t_max = 10.0
t = 0.0
dt = 0.025
while t <= t_max:
    append1 = []
    append2 = []
    for i in range(len(PSIS[0])):
        append1.append(-1*c1*PSIS[0][i]*sin(t*EIGEN[0]*0.5)-1*c2*PSIS[1][i]*sin(t*EIGEN[1]*\
        0.5)-1*c3*PSIS[2][i]*sin(t*EIGEN[2]*0.5)-1*c4*PSIS[3][i]*sin(t*EIGEN[3]*0.5))
        append2.append(c1*PSIS[0][i]*cos(t*EIGEN[0]*0.5)+c2*PSIS[1][i]*\
        cos(t*EIGEN[1]*0.5)+c3*PSIS[2][i]*cos(t*EIGEN[2]*0.5)+c4*PSIS[3][i]*cos(t*EIGEN[3]*0.5))
    PSI_Im.append(append1)
    PSI_Re.append(append2)
    t += dt
j = 0

### Animating the time evolution

In [ ]:
scene = vp.canvas()
while j< len(PSI_Re):
    graphlist1 = [(x[j],PSI_Re[j][0])]
    graphlist2 = [(x[j],PSI_Im[j][0])]
    graphlist3 = [(x[j],PSI_Re[j][0]**2 + PSI_Im[j][0]**2)]
    vp.rate(20)
    psiimag.delete()
    psireal.delete()
    psisquare.delete()
    for r in range(len(x)):
        graphlist2.append((x[r],PSI_Im[j][r]))
        graphlist1.append((x[r],PSI_Re[j][r]))
        graphlist3.append((x[r],PSI_Re[j][r]**2 + PSI_Im[j][r]**2))
    psireal = vp.gcurve(color = vp.color.red)
    psiimag = vp.gcurve(color = vp.color.green)
    psisquare = vp.gcurve(color = vp.color.blue)
    psiimag.plot(pos = graphlist2)
    psireal.plot(pos = graphlist1)
    psisquare.plot(pos = graphlist3)
    j+=1